In [1]:
import cv2
from scipy.spatial import distance as dist

def get_mm_per_pixel_size(image):
    # Known physical size of the ArUco marker (in centimeters)
    physical_marker_size = 20  # Replace with your measurement

    # Load the image containing the ArUco marker
    # image = cv2.imread('aruco_marker_image.jpg')
    # Detect and identify the ArUco marker
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
    parameters = cv2.aruco.DetectorParameters_create()
    corners, ids, _ = cv2.aruco.detectMarkers(image, aruco_dict, parameters=parameters)

    if ids is not None:
        # Calculate the size of the ArUco marker in pixels
        marker_size_in_pixels = max(corners[0][0][2][0] - corners[0][0][0][0], 
                                    corners[0][0][1][1] - corners[0][0][0][1])

        # Calculate the size of a pixel in real-world units
        mm_per_pixel_size = physical_marker_size / marker_size_in_pixels

        print(f"Size of a pixel: {mm_per_pixel_size} mm/pixel")
    else:
        print("ArUco marker not found in the image.")
    
    return mm_per_pixel_size
        
image = cv2.imread('aruco_marker_image.jpg')

mm_per_pixel_size = get_mm_per_pixel_size(image)

Size of a pixel: 0.03861003861003861 mm/pixel


In [2]:

def get_length_dmax(image, mm_per_pixel_size):
    image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # convert to grayscale for threshold
    cv2.imwrite("Grayscale.jpg", image_grayscale)
    ret, thresholded = cv2.threshold(image_grayscale, 40, 180, cv2.THRESH_BINARY)
    image_mask = cv2.adaptiveThreshold(thresholded, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 59, 19)
    cv2.imwrite("Mask.jpg", image_mask)
    image_dilate = cv2.dilate(image_mask, None, iterations=2)
    cv2.imwrite("Dilate.jpg", image_dilate)
    image_erode = cv2.erode(image_dilate, None, iterations=2)
    cv2.imwrite("Erode.jpg", image_erode)

    # Find contours in the image
    contours, _ = cv2.findContours(image_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)

    # Draw a bounding box around the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 255), 3)
    cv2.imwrite("bounding_box.jpeg", image)
    
    # Calculate the coordinates of the four corners of the bounding box
    top_left = (x, y)
    top_right = (x + w, y)
    bottom_right = (x + w, y + h)
    bottom_left = (x, y + h)

    length = dist.euclidean(top_left, top_right) * mm_per_pixel_size
    dmax = dist.euclidean(top_left, bottom_left) * mm_per_pixel_size
    print(f"The Complete len is --> {length}")
    print(f"The max diameter is --> {dmax}")
    print(f"cd:{0.3}")
    # sys.stdout.flush()


    print(top_left, top_right, bottom_right, bottom_left)

    return length, dmax, top_left, top_right, bottom_right, bottom_left

length, dmax, top_left, top_right, bottom_right, bottom_left = get_length_dmax(image, mm_per_pixel_size)

The Complete len is --> 132.85714285714286
The max diameter is --> 10.501930501930502
cd:0.3
(292, 1498) (3733, 1498) (3733, 1770) (292, 1770)


In [3]:
for i in range(1):
    image = cv2.imread('aruco_marker_image.jpg')
    mm_per_pixel_size = get_mm_per_pixel_size(image)
    length, dmax, top_left, top_right, bottom_right, bottom_left = get_length_dmax(image, mm_per_pixel_size)

Size of a pixel: 0.03861003861003861 mm/pixel
The Complete len is --> 132.85714285714286
The max diameter is --> 10.501930501930502
cd:0.3
(292, 1498) (3733, 1498) (3733, 1770) (292, 1770)


In [11]:
def getDMin(image):
    image_gaussian_blur = cv2.GaussianBlur(image, (5, 5), cv2.BORDER_DEFAULT)
    image_canny = cv2.Canny(image_gaussian_blur, 50, 200, True)
    image_dilate = cv2.dilate(image_canny, None, iterations=3)
    # image_erode = cv2.erode(image_dilate, None, iterations=2)

    image_gray = cv2.cvtColor(image_dilate, cv2.COLOR_BGR2GRAY)
    (thresh, binary) = cv2.threshold(image_gray, 127, 255, cv2.THRESH_BINARY)

    print(binary.shape)  # (height, width, channels)
    # height, width = binary.shape
    
getDMin(image)

error: OpenCV(4.5.5) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xa96199bf::Set<1,-1,-1>,struct cv::impl::A0xa96199bf::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [12]:
import cv2 as cv

def detectMinGruve(croppedImgFile):
    imgFile = croppedImgFile
    # imgFile = cv.imread(f"{saveFolder}\cropped_img.jpg")
    ## ------------------------------Manipulation starts here----------------------
    gray = cv.cvtColor(imgFile, cv.COLOR_BGR2GRAY)  # convert to grayscale for threshold
    imgBlur = cv.medianBlur(gray, 5)
    # or try GaussianBlur
    
    imgBlur = cv.GaussianBlur(imgFile, (5, 5), cv.BORDER_DEFAULT)
    blurPath = f"{saveFolder}/blurimg.jpg"

    cv.imwrite(blurPath, imgBlur)
    imgCanny = cv.Canny(imgBlur, 50, 200, True)
    
    imgDial = cv.dilate(imgCanny, None, iterations=3)
    imgErode = cv.erode(imgDial, None, iterations=2)

    print("Saving manupilated img")
    # cv.imshow("Manipulated", imgErode)
    cv.imwrite(savePath, imgDial)
    
    print(f"cd:{0.3}")
    sys.stdout.flush()

    ##----------------------------------Converting to Binary---------------------------------

    prcImg = cv.imread(savePath)
    # prcImg = imgDial
    prcGray = cv.cvtColor(prcImg, cv.COLOR_BGR2GRAY)
    (thresh, binary) = cv.threshold(prcGray, 127, 255, cv.THRESH_BINARY)

    print(binary.shape)  # (height, width, channels)
    height, width = binary.shape
    cv.imwrite(f"{saveFolder}/Binary.jpg", binary)
    ## ---------------------------------Binary Ends here-------------------------------------

    ## ---------------------------------Manipulation ends here ------------------------------

    ## --------------------------------Detecting the points ---------------------------------

    UpperList = [None] * width
    LowerList = [None] * width
    start = time.time()


    for x in range(0, (width - 1)):
        yLower = height - 1
        yUpper = 0
        while yUpper < (height * 0.66):
            if binary[yUpper, x] == 255:
                UpperList[x] = yUpper
                break
            else:
                yUpper += 2

        while yLower > height * 0.33:
            if binary[yLower, x] == 255:
                LowerList[x] = yLower
                break
            else:
                yLower -= 2


    filterUpper = list(filter(lambda x: x is not None, UpperList))
    filterLower = list(filter(lambda x: x is not None, LowerList))

    Upperdf = pd.DataFrame(filterUpper, columns=["Upper"])
    Lowerdf = pd.DataFrame(filterLower, columns=["Lower"])
    
    startY = Upperdf[50:150].mean(axis=0)
    endY = Lowerdf[50:150].mean(axis=0)
    startY = int(startY)
    endY = int(endY)
    end = time.time()

    print("Time: ", end - start)

    print("Upper Pixel", startY)
    print("Lower Pixel", endY)

    ## -------------------------------------Detecting Points Ends here-----------------------------

    ## -------------------------------------Plotting the points------------------------------------

    plotimg = cv.imread(imgPath)
    # plotimg = fullImgFile


    # selectUp.to_csv("out/csv/selectUp.csv")
    # selectDown.to_csv("out/csv/selectDown.csv")
    
    start = (150, startY)
    end = (150, endY)
    the_height = dist.euclidean(start, end) / pixelsPerMetric
    print(f"The min height is --> {the_height}")
    min_height = the_height
    

    cv.line(plotimg, (0, startY), (width, startY), (0, 0, 255), 2)
    cv.line(plotimg, (0, endY), (width, endY), (0, 255, 0), 2)

    cv.imwrite(f"{saveFolder}/line.jpg", plotimg)
    
    print(f"cd:{0.4}")
    sys.stdout.flush()

    ## -------------------------------------Plotting Ends here---------------------------------

detectMinGruve(image)

NameError: name 'saveFolder' is not defined